# Target
---------------------

__Overview:__ 2 sets of data in this 'kernels only' competition: News and Prices/Returns. Use both sets to predict the movement of a given financial asset in the next 10 days. _I believe it is a 10 day residual expected return. So this is one numeric prediction for each asset in the universe (??)_

We have data from 2007 to 2017 for training and must predict the movement of assets from Jan 2017 to July 2019.

This will detail the entire project so far

We've actually done quite a bit of work, but it is so disjointed that it is nearly impossible to progress
in any meaningful way

Need to Organize

__SEE RESEARCH at bottom for links and additional notes__

### TOC
---------------

__Scripts folder:__ 

1. Sigma_imports.py  -> Not updated
2. AssetExpand.py -> left an old func; not yet tested in kernel
3. sigma_1.py -> func for checking mem usage and submitting
4. sigma_plot -> 

__Notebooks folder:__

1. Output folder -> random output: text files, csvs

2. Lightgbm_basic -> Attempts to use demos to run a small scale model. This obviously fails since the dates do not allow for a JOIN. Invested good amount of time trying to produce a small sample set that can be explored locally rather than on the kernel. The sample set exists, but is inaccurate. It should only be used in preprocessing.

3. feature_analysis -> so far only type and value counts. Could perform EDA and correlation (filter methods)

4. processing_speed -> Everything big data. Keep that kernel from crashing

5. News_working (link with scratch_news) -> statsmodel (need to move), expandinging features into indicators, aggregation of expansion

6. scratch_news -> Indicators, freqs, word vecs, DFS

7. scratch_market -> a small sample is produced for experimentation. Additionally, market datasets pulled from datareader.


__References folder:__

1.

__Notes:__ next day. Continue to develop sample and organize/abstract. 

Start to divide work between market and news. You can take large intervals of market data from data_reader and conduct analysis. In fact, you could entirely focus on the market data for a few days - it will benefit you in the interview process.   

Think that's the best idea. Incorporate everything you've been working on, possibly BM as well. Pull the data and conduct backtests. The main holdup is the target variable. Consider addressing that first.  

1. Research the market residual. 
2. Identify relevant market data.
3. Focus exclusively on market data and begin submitting predictions

__UNIVERSE__  
Jury is still out. You should probably train on all. When you set your metric to track progress, the metric will use the universe feature for your score.


__Target: returnsOpenNextMktres10__   
market-residualized return. This is the TARGET VARIABLE used in competition scoring. The market data has been filtered such that returnsOpenNextMktres10 is always not null.  

market-residualized (Mktres), meaning that the movement of the market as a whole has been accounted for, leaving only movements inherent to the instrument.

In [1]:
path = '../scripts'
import sys
sys.path.append(path)
from sigma_imports import *
from sigma_code import *

data_path = '/Users/jacob/Desktop/docs/kaggle/_2_sigma_news/_g_data/data'
df_market = pd.read_csv(data_path + '/marketdata_sample.csv') 
df_news = pd.read_csv(data_path + '/news_sample.csv')

### Target review

You are trying to predict a confidene value $y_t \in [-1, 1]$ This value is a strength measure of the models prediction that the 10 day rez will be positive or negative. Therefore, this is _not_ a regression. But rather a classifier using probabilities of binary outcomes. The best way to understand this is by example:  

For t = 3 and asset returns $[r_1, r_2, r_3, r_4]$ say my model predicts ```[-1, 1, 1, -1]``` for the target. The model, however, is only predicting $[0,1]$ s.t. any probability $\ge .5 = 1$ and = 0 o.w.   
1: positive return and 0: negative return.  

So lets say the strength of those prediction (confidence predictions: $p_{3i}$) are as follows ```[0.3, 0.7, 0.8, 0.1]```. Therefore, the model is more/less confident on certain predictions. Now, say the target values are ```[-.005, .002, .03, .06]```, notice that the model missed a pred that it was fairly confident about. It was 90% confident that the return was going to be negative. Not only was it positive, but it was positive by a lot. This should hurt the score a lot more therefore. Let's also say that one of the assets (4) is not in the daily universe. 

For each day in the evaluation time period, we calculate:
$$x_{t} = \sum_{i}\hat y_{ti}r_{ti}u_{ti}$$  

where $r_{ti}$ is the 10-day market-adjusted leading return for day $t$ for instrument $i$, and $u_{ti}$ is a 0/1 `universe` variable (see the data description for details) that controls whether a particular asset is included in scoring on a particular day.

For $\hat{y}$, we need to perform the transformation of the confidence prediction: $\hat{y}_{ti} = (p_{ti} * 2) - 1$. This allows for the following.  
If the prediction is:  
True positive (TP): $\hat{y}_{ti} \in \mathbb{R}^+$ and $\hat{y}_{ti}r_{ti} \in \mathbb{R}^+$  Since $p_{ti} > .5$  
True Negative (TN): $\hat{y}_{ti} \in \mathbb{R}^-$ and $\hat{y}_{ti}r_{ti} \in \mathbb{R}^+$  Since $p_{ti} <.5$  

Both results improve your score by a weight determined by the confidence and actual return.  
FP and FN then take on weighted negative values

So our $\hat{y}_{3i}$ = ```[-0.4, 0.4, 0.6, -0.6]``` 

Cool. Let's plug in some values:

$x_{3} = \sum_{i=1}^4\hat y_{3i}r_{3i}u_{3i} = (-.04 * -.005 * 1) +   (0.4 * .002 * 1) + (0.6 * 0.03 * 0) + (-0.6 * 0.06 * 1)\\ 
\qquad = (0.0002 * 1) + (0.0008 * 1) + (0) + (-0.036 * 1)\\ 
\qquad = -0.035
$  
Ouch! That missed prediction and universe condition really hurt.

Then all of the daily scores are averaged 

Your submission score is then calculated as the mean divided by the standard deviation of your daily $x_t$ values:

$$score = \frac{\bar x_{t}}{\sigma(x_{t})}.$$

Of course, higher scores are desireble, so less variation between daily scores is important. 



In [16]:
# TRYING TO PREVENT DATE TYPE CONVERSION
# THIS SHOULD BE UPDATED FROM PANDA DATAREADER
data = pd.read_csv(data_path + '/sigma_data.csv', index_col='Date', parse_dates=True, 
                 infer_datetime_format=True)

In [17]:
data.head()

,asset,Open,Close,Volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,returns_open_raw10_next
Date,,,,,,,,,
2016-01-19,CHL,52.790001,52.570000,1180200.0,0.037009,0.039607,-0.046641,-0.036273,-0.034444
2016-01-19,WMT,62.250000,62.560001,13051300.0,0.010121,0.011309,0.017740,0.028515,-0.078002
2016-01-19,NGG,73.569870,73.755455,692700.0,0.009518,0.000297,-0.020221,-0.020998,-0.054715
2016-01-19,AUO,2.660000,2.640000,796100.0,0.011429,0.011342,-0.055263,-0.065478,0.046162
2016-01-19,SIEB,1.180000,1.150000,2300.0,-0.059089,-0.033336,-0.114880,-0.089128,0.017094


## Residual

https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/

We'll consider seasonal to be market. Frequency is set to the number of assets. This should end up giving daily residuals

In [18]:
import statsmodels.tsa.api as tsa
from statsmodels.tsa.seasonal import seasonal_decompose 

x = data[['returns_open_raw10_next']]
# Freq will depend on asset number - this needs to be researched
result = seasonal_decompose(x, model='additive', freq=50)

data['returns_res10_next'] = result.resid
# Lose the first and last day
data = data[-data['returns_res10_next'].isnull()]

In [19]:
df = data.copy()

In [20]:
df['date'] = df.index.strftime("%Y%m%d").astype(int)
df['universe'] = universe_feat(df)
df.reset_index(drop=True, inplace=True)

# Needed for dataframe splits
dates = df['date'].unique()

# Convert targets to binary
target = ['returns_res10_next']
label = ['label']
# df[target[0]] = (df[target[0]] > 0).astype(int)
df[label[0]] = (df[target[0]] > 0).astype(int)
# Consider vectorize - later

def train_feats(dframe, drops=list):
    # could do list comp
    train_feat = []
    for col in dframe.columns:
        if col not in drops:
            train_feat.append(col)
    return train_feat

feats = train_feats(df, drops=['date', 'asset', 'universe', 
                               'returns_open_raw10_next', 'returns_res10_next',
                               'label'])

train_range = range(len(dates))[:int(0.70*len(dates))]
val_range = range(len(dates))[int(0.70*len(dates)):int(0.9*len(dates))]
test_range = range(len(dates))[int(0.9*len(dates)):]

import lightgbm as lgb

# KINDA LOOKS CRAPPY. THIS JUST SETS THE TRAINING/VAL SETS WITH PREDICTORS
# train data - numpys
X_train = df[feats].fillna(0).loc[df['date'].isin(dates[train_range])].values
Y_train = df[label].fillna(0).loc[df['date'].isin(dates[train_range])].values

# GETS INTERESTING -> INPUTS DATA INTO INSTANCE
lgb_train = lgb.Dataset(X_train, Y_train[:,0])
print(X_train.shape, Y_train.shape)

# validation data - numpys
X_v = df[feats].fillna(0).loc[df['date'].isin(dates[val_range])].values
Y_v = df[label].fillna(0).loc[df['date'].isin(dates[val_range])].values

lgb_val = lgb.Dataset(X_v, Y_v[:,0]),
print(X_v.shape, Y_v.shape)

# test data
X_test = df[feats].fillna(0).loc[df['date'].isin(dates[test_range])].values
Y_test = df[label].fillna(0).loc[df['date'].isin(dates[test_range])].values

print(X_test.shape, Y_test.shape)

param = {"objective" : "binary",
          "metric" : "binary_logloss",
          "verbosity" : -1 }

# TONS OF HYPERPARAMENTERS. THIS FIT FUNC USES THE VAL SET TO TRAIN AND 
# SELECT THE BEST ITERATION

model = lgb.train(param, lgb_train, valid_sets=lgb_val, 
                  early_stopping_rounds=10) 
                  

(8022, 7) (8022, 1)
(2396, 7) (2396, 1)
(1125, 7) (1125, 1)
[1]	valid_0's binary_logloss: 0.693885
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.694804
[3]	valid_0's binary_logloss: 0.695783
[4]	valid_0's binary_logloss: 0.695499
[5]	valid_0's binary_logloss: 0.695503
[6]	valid_0's binary_logloss: 0.696995
[7]	valid_0's binary_logloss: 0.700037
[8]	valid_0's binary_logloss: 0.700629
[9]	valid_0's binary_logloss: 0.70292
[10]	valid_0's binary_logloss: 0.703661
[11]	valid_0's binary_logloss: 0.70439
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.693885


## Method 1

In [21]:
def custom_metric(date, pred_proba, num_target, universe):
    y = pred_proba*2 - 1
    r = num_target#.clip(-1,1) # get rid of outliers - not sure about this
    # I see the logic, but you'd need to remove the corresponding preds, unv
    x = y * r * universe
    result = pd.DataFrame({'day' : date, 'x' : x})
    x_t = result.groupby('day').sum().values
    return np.mean(x_t) / np.std(x_t)

In [22]:
# FULL TEST SET DATASET VERSION
preds = model.predict(X_test, num_iteration=model.best_iteration) 
date_vec = df['date'].loc[df['date'].isin(dates[test_range])]
u = df['universe'].loc[df['date'].isin(dates[test_range])]
actual = df[target].fillna(0).loc[df['date'].isin(dates[test_range])].values[:, 0]


custom_metric(date_vec, preds, actual, u)

0.6416903792259109

In [8]:
# MICRO VERSION
X = df[feats].fillna(0).loc[df['date'].isin([20161114, 20161115])].values
preds = model.predict(X, num_iteration=model.best_iteration) 
actual = df[target].fillna(0).loc[df['date'].isin([20161114, 20161115])].values
u = df['universe'].fillna(0).loc[df['date'].isin([20161114, 20161115])].values
date_vec = df['date'].loc[df['date'].isin([20161114, 20161115])].values

custom_metric(date_vec, preds, actual[:, 0], u)

0.5400366074034371

In [9]:
# WALKTHROUGH
y = preds*2 - 1
r = actual[:,0]   
x = y*r*u
result = pd.DataFrame({'day' : date_vec, 'x' : x})
x_t = result.groupby('day').sum().values
np.mean(x_t) / np.std(x_t)

0.5400366074034371

## Method 2

In [ ]:
total = []
for day in test_range:
# for day in range(214, 216):
    print(dates[day])
    
    X = df[feats].fillna(0).loc[df['date'] == dates[day]].values
    u = df['universe'].fillna(0).loc[df['date'] == dates[day]].values
    # use target not label
    r = df[target].fillna(0).loc[df['date'] == dates[day]].values
    
    y = model.predict(X, num_iteration=model.best_iteration)*2 - 1
    # shape needs to be adjusted. This corrects for universe
    r_u = r[:, 0]*u
    
    xt = r_u.T @ y
    
    print(xt)
    total.append(xt)
    
    


In [11]:
# SHOULD BE 24 days
print(x.shape)
len(set(df.date))
# test data
X_test_df = (
            df.
            fillna(0).
            loc[df['date'].
            isin(dates[test_range])]
)
len(set(X_test_df.date))
# Confirmed

(100,)


24

In [12]:
np.mean(total) / np.std(total)
# Pretty good? I don't trust it then. Of course, this is over a small test 
# interval. So the model is probably getting lucky. Plus, this is not resisualized


0.8724795888022177

### Notes

Look to implement this  
```from sklearn.model_selection import TimeSeriesSplit```




### Research


Links to check out:

https://www.kaggle.com/nareyko/fast-lags-calculation-concept-using-numpy-arrays

Fundamental data?  

https://pypi.org/project/wallstreet/0.1.5/

https://alphascientist.com/feature_selection.html

Read this first - sound very close to a market residualized
https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/
